In [1]:
import _init_paths
from pycocotools.coco import COCO
import config as C
import gensim



coco = COCO('/media/zawlin/ssd/coco/annotations/instances_train2014.json')
cats = coco.loadCats(coco.getCatIds())
coco_classes=[cat['name'] for cat in cats]

loading annotations into memory...


Done (t=9.76s)
creating index...


index created!


In [21]:
model = gensim.models.Word2Vec.load(C.word2vec_wiki_path)
model_news = gensim.models.Word2Vec.load_word2vec_format(C.word2vec_news_path,binary=True)

In [2]:
cats = coco.loadCats(coco.getCatIds(['cat']))
print cats

[{u'supercategory': u'animal', u'id': 17, u'name': u'cat'}]


In [65]:
import utils.zl_utils as zl
import numpy as np
voc_v = {}
coco_v = {}

voc_classes = ['aeroplane', 'bicycle', 'bird', 'boat',
'bottle', 'bus', 'car', 'cat', 'chair',
'cow', 'dining table', 'dog', 'horse',
'motorbike', 'person', 'potted plant',
'sheep', 'sofa', 'train', 'tv monitor']
for n in voc_classes:
    ns = n.split(' ')
    l = []
    for ni in ns:
        l.append(model[ni])    
    voc_v[n]=np.average(np.array(l),axis=0)    
    voc_v[n] =  voc_v[n]/np.linalg.norm(voc_v[n])
    #voc_v[n] =  voc_v[n]/np.sum(voc_v[n])
    
    
for n in coco_classes:
    ns = n.split(' ')
    l = []
    for ni in ns:
        l.append(model[ni])        
    coco_v[n]=np.average(np.array(l),axis=0)
    coco_v[n] =  coco_v[n]/np.linalg.norm(coco_v[n])
    #coco_v[n] =  coco_v[n]/np.sum(coco_v[n])

In [98]:
from sklearn.decomposition import sparse_encode
from sklearn.decomposition import SparseCoder
D = []
idx2word = []
for i in xrange(len(voc_v.keys())):
    idx2word.append(voc_v.keys()[i])
    D.append(voc_v[voc_v.keys()[i]])
idx2word = np.array(idx2word)
D = np.array(D)

coder = SparseCoder(dictionary=D, transform_n_nonzero_coefs=5,
                        transform_alpha=None, transform_algorithm='lars')

for k in coco_v:
    y = coco_v[k]
    x  = coder.transform(y.reshape(1,-1))[0]
    idx = np.argsort(x)[::-1]
    output_str = '%s = '%k
    for i in idx:
        if x[i]>0:
            output_str = '%s%f*%s+ ' %(output_str,x[i],idx2word[i])
    print output_str[:-2]

toilet = 0.228165*dining table+ 0.144676*bottle+ 0.142471*sofa+ 0.028060*potted plant+ 0.017124*dog
teddy bear = 0.184379*dog+ 0.111172*cat+ 0.087568*bird+ 0.062930*bottle+ 0.037765*sofa
cup = 0.212292*bottle+ 0.054781*dining table+ 0.031324*cow+ 0.024453*potted plant+ 0.014748*person
bicycle = 1.000000*bicycle
kite = 0.278348*bird+ 0.096703*aeroplane+ 0.091577*boat+ 0.051782*bicycle
carrot = 0.242650*potted plant+ 0.175661*cow+ 0.064328*bottle+ 0.011625*dog
stop sign = 0.077540*bus+ 0.065602*person+ 0.027401*train+ 0.016645*tv monitor+ 0.000152*car
tennis racket = 0.143779*dining table+ 0.041824*motorbike+ 0.029311*bicycle+ 0.010543*bird+ 0.007086*potted plant
donut = 0.129756*bottle+ 0.082033*sofa+ 0.071587*dog+ 0.032196*car+ 0.020845*cow
snowboard = 0.157547*bicycle+ 0.051890*motorbike+ 0.009728*dog+ 0.008964*chair+ 0.001925*dining table
sandwich = 0.117259*cow+ 0.106507*bottle+ 0.065147*dog+ 0.044265*dining table+ 0.007274*sofa
motorcycle = 0.316220*motorbike+ 0.302444*bicycle+ 0.1

In [100]:
from sklearn.decomposition import sparse_encode
from sklearn.decomposition import SparseCoder
D = []
idx2word = []
for i in xrange(len(voc_v.keys())):
    idx2word.append(voc_v.keys()[i])
    D.append(voc_v[voc_v.keys()[i]])
idx2word = np.array(idx2word)
D = np.array(D)

coder = SparseCoder(dictionary=D, transform_n_nonzero_coefs=5,
                        transform_alpha=None, transform_algorithm='lars')
coco_recon = {}
classes = tuple(['__background__'] + [c['name'] for c in cats])
class_to_ind = dict(zip(_classes, xrange(81)))
class_to_coco_cat_id = dict(zip([c['name'] for c in cats], coco.getCatIds()))
coco_v_i = {}
coco_recon_i = {}
for k in coco_v:
    coco_v_i[class_to_ind[k]] = coco_v[k]
    y = coco_v[k]
    x  = coder.transform(y.reshape(1,-1))[0]
    x =  x/np.sum(x)
    coco_recon[k] = x
    coco_recon_i[class_to_ind[k]] = coco_recon[k]
    idx = np.argsort(x)[::-1]
    output_str = '%s = '%k
    for i in idx:
        if x[i]>0:
            output_str = '%s%f*%s+ ' %(output_str,x[i],idx2word[i])

zl.save('/media/zawlin/ssd/iccv2017/_cache/coco_recon.pkl',coco_recon)
zl.save('/media/zawlin/ssd/iccv2017/_cache/coco_v.pkl',coco_v)

zl.save('/media/zawlin/ssd/iccv2017/_cache/coco_recon_i.pkl',coco_recon_i)
zl.save('/media/zawlin/ssd/iccv2017/_cache/coco_v_i.pkl',coco_v_i)

In [102]:
#print coco_v_i[1]
#print coco_v['person']

[  1.52660953e-02  -3.44870798e-02   3.33808064e-02   4.17993590e-03
   3.82449180e-02   7.49640446e-03   2.14189999e-02   3.82512249e-02
   6.39715642e-02  -2.19673514e-02   6.82509914e-02  -4.54244539e-02
  -7.67499348e-03   3.35038491e-02  -5.87580353e-02  -1.31457150e-02
   9.69343455e-05   3.87313999e-02  -3.16445194e-02   3.53454724e-02
  -2.42871679e-02   1.60894040e-02   1.24808662e-02  -8.26289319e-03
   5.25817508e-03  -1.77298021e-02  -6.69250563e-02  -3.44858170e-02
  -5.72726764e-02   3.40193603e-03  -1.22423712e-02   5.63055202e-02
  -2.57999590e-03  -8.40061996e-03   1.26408860e-02  -2.45957002e-02
  -8.69039388e-04   1.38257835e-02   4.62425835e-02  -3.32270041e-02
  -1.71554163e-02   8.05647019e-03   8.22205469e-03  -3.73440306e-03
  -2.85277721e-02   2.74068192e-02   3.06828693e-02  -2.30959263e-02
  -2.54589729e-02   1.75976995e-02  -1.45412944e-02  -1.92564353e-02
  -4.14335653e-02  -5.57133928e-02   4.20543039e-03  -1.67029686e-02
   1.10730371e-02   1.98141877e-02

In [96]:
classes = tuple(['__background__'] + [c['name'] for c in cats])
class_to_ind = dict(zip(_classes, xrange(81)))
class_to_coco_cat_id = dict(zip([c['name'] for c in cats], coco.getCatIds()))

In [97]:
print classes
print class_to_ind
print class_to_coco_cat_id

('__background__', u'person', u'bicycle', u'car', u'motorcycle', u'airplane', u'bus', u'train', u'truck', u'boat', u'traffic light', u'fire hydrant', u'stop sign', u'parking meter', u'bench', u'bird', u'cat', u'dog', u'horse', u'sheep', u'cow', u'elephant', u'bear', u'zebra', u'giraffe', u'backpack', u'umbrella', u'handbag', u'tie', u'suitcase', u'frisbee', u'skis', u'snowboard', u'sports ball', u'kite', u'baseball bat', u'baseball glove', u'skateboard', u'surfboard', u'tennis racket', u'bottle', u'wine glass', u'cup', u'fork', u'knife', u'spoon', u'bowl', u'banana', u'apple', u'sandwich', u'orange', u'broccoli', u'carrot', u'hot dog', u'pizza', u'donut', u'cake', u'chair', u'couch', u'potted plant', u'bed', u'dining table', u'toilet', u'tv', u'laptop', u'mouse', u'remote', u'keyboard', u'cell phone', u'microwave', u'oven', u'toaster', u'sink', u'refrigerator', u'book', u'clock', u'vase', u'scissors', u'teddy bear', u'hair drier', u'toothbrush')
{u'toilet': 62, u'teddy bear': 78, u'cup